## Para descargar los PDFs

In [1]:
import os
import requests
from bs4 import BeautifulSoup
import re

In [10]:
def download_file(url):
    local_filename = url.split('/')[-1]
    # NOTE the stream=True parameter below
    with requests.get(url, stream=True) as r:
        r.raise_for_status()
        with open(local_filename, 'wb') as f:
            for chunk in r.iter_content(chunk_size=8192): 
                if chunk: # filter out keep-alive new chunks
                    f.write(chunk)
                    # f.flush()
    print(local_filename)
    return local_filename

In [11]:
DAILY_FOLDER = "./pdf_files/"
ES_REPORT_URL = "https://www.mscbs.gob.es/profesionales/saludPublica/ccayes/alertasActual/nCov-China/situacionActual.htm"
PDF_BASE_URL = "https://www.mscbs.gob.es/profesionales/saludPublica/ccayes/alertasActual/nCov-China/"

In [12]:
pdf_paths = get_file_names(ES_REPORT_URL)

In [13]:
def get_file_names(url):
    try:
        req_page = requests.get(url, verify=True)
    except Exception as e:
        raise Exception(f"Could not get web page content: {e}")
    
    re_pdf = re.compile(r"(documentos/.+?\.pdf)")

    pdf_paths = list(set(re_pdf.findall(req_page.text)))
    return pdf_paths

In [14]:
pdf_paths = get_file_names(ES_REPORT_URL)
pdf_paths = [PDF_BASE_URL+i for i in pdf_paths if "Informacion_inicial_alerta" not in i]

In [15]:
for pdf_path in pdf_paths:
        pdf_name = pdf_path.split("/")[-1]
        pdf_path_get = requests.get(pdf_path)
        with open(
            os.path.join(DAILY_FOLDER, pdf_name),
            'wb'
        ) as f:
            f.write(pdf_path_get.content)

In [19]:
for i in range(64,80):
    path = 'https://www.mscbs.gob.es/profesionales/saludPublica/ccayes/alertasActual/nCov-China/documentos/Actualizacion_{}_COVID-19.pdf'.format(i+1)
    download_file(path)

Actualizacion_65_COVID-19.pdf


HTTPError: 404 Client Error: Not found for url: https://www.mscbs.gob.es/profesionales/saludPublica/ccayes/alertasActual/nCov-China/documentos/Actualizacion_66_COVID-19.pdf

In [21]:
lst_files = [file for file in os.listdir() if 'Act' in file]
if os.path.isdir("./pdf_data/"):
    for f in lst_files:
        os.rename("./{}".format(f), "./pdf_files/{}".format(f))
else:
    os.mkdir("./pdf_data/")
    for f in lst_files:
        os.rename("./{}".format(f), "./pdf_files/{}".format(f))

In [18]:
import tabula
from tabula import read_pdf
df = tabula.read_pdf("pdf_files/Actualizacion_65_COVID-19.pdf", stream=True, pages=1)[0] 
df.columns = ['Comunidades Autónomas', 'Casos totales', 'IA','Hospitalizados', 'UCI', 'Fallecidos', 'Curados', 'Nuevos casos (24h)']

In [23]:
from datetime import datetime, timedelta
from datetime import date
date_today= date.today().strftime('%d/%m')
date_today

'04/04'

In [19]:
df = df[['Comunidades Autónomas', 'Casos totales', 'Fallecidos', 'Hospitalizados', 'Curados', 'Nuevos casos (24h)']]
df = df.iloc[4:23]

#Quitar símbolos raros qye nos sobran
df['Comunidades Autónomas'] = df['Comunidades Autónomas'].str.replace('¥', '')
df['Casos totales'] = df['Casos totales'].str.replace('¥', '')
df['Hospitalizados'] = df['Hospitalizados'].str.replace('¥', '')
df['Fallecidos'] = df['Fallecidos'].str.replace('¥', '')
df['Curados'] = df['Curados'].str.replace('¥', '')
df['Nuevos casos (24h)'] = df['Nuevos casos (24h)'].str.replace('¥', '')
df = df.set_index('Comunidades Autónomas')
df


,Casos totales,Fallecidos,Hospitalizados,Curados,Nuevos casos (24h)
Comunidades Autónomas,,,,,
Andalucía,7.869,426,3.896,463,495
Aragón,3.078,251,1.505,440,189
Asturias,1.522,76,766,190,89
Baleares,1.271,71,634,422,14
Canarias,1.564,78,632,123,74
Cantabria,1.384,68,619,99,63
Castilla La Mancha,9.324,989,3.133,657,801
Castilla y León,8.332,786,2.664,2.021,457
Cataluña,24.734,2.508,17.678,8.635,1.274


In [ ]:

# df['Comunidades Autónomas'] = df['Comunidades Autónomas'].str.replace(',', '')

# pass them to df.replace(), specifying each char and it's replacement:
#
# df[cols] = df[cols].replace({'\$': '', ',': ''}, regex=True)
df